# Comparison star generator

## For information about your TESS target

1. Go to https://exofop.ipac.caltech.edu/tess/
2. Enter the TIC number in the TIC search box
2. Use the RA/Dec for the `ra` and `dec` variables in the cell below.
3. Use the "TESS mag" for the variable `Cmag` in the cell below.
4. Use the "depth" for the variable `depth` in the cell below.

In [ ]:
import functools
import re
from pathlib import Path
import requests

import numpy as np

from astropy import units as u
from astropy.table import Table
from astropy.coordinates import SkyCoord

from stellarphot.visualization.comparison_functions import (
    read_file, set_up, match, mag_scale, in_field, wrap, make_markers,
    viewer
)
from stellarphot.visualization.seeing_profile_functions import set_keybindings

## Change the settings in this cell as needed

In [ ]:
directory_with_images = '..'

# 👉 This could be any image from roughly the middle of the sequence.
sample_image_for_finding_stars = 'TIC_237205154.01-S001-R013-C001-rp.fit'

# 👉 You will be able to use this name later to pull out the data you want
object_of_interest = 'TIC_237205154.01'

# 👉 Get these variable from ExoFOP-TESS or Exoplanet ETD
 
ra_object = '19:34:19.97'
dec_object = '76:39:39.91'

Cmag = 13.0024 # 👉 Use TESS magnitude from ExoFOP-TESSfor this
depth = 16908.821 / 1000 # 👉 Depth should be in parts per thousand, ExoFOP gives it in ppm.

bright_mag_limit = 9
dim_mag_limit = 16

# 👉 File to save apertures in
aperture_output_file = 'aperture_locations.fits'

TESS_target = True  # change to False (capitalized) if this is NOT a TESS object

if TESS_target:
    server = "https://www.astro.louisville.edu"
    gaia_aperture_url = server + f"/cgi-bin/gaia_to_aij/upload_request.cgi?ra={ra_object}&dec={dec_object}&mag={Cmag}&depth={depth}"

    result = requests.get(gaia_aperture_url)
    links = re.search('href="(.+)"', result.text.replace('\n', ''), )

    target_file_contents = requests.get(server + links[1])
    with open('gaia_stars.radec', 'wt') as f:
        f.write(target_file_contents.text)
    targets_from_file = read_file('gaia_stars.radec')
else:
    targets_from_file = []

In [ ]:
coordinate = SkyCoord(ra=ra_object, dec=dec_object, unit=("hour", "degree"))

In [ ]:
ccd, vsx = \
    set_up(sample_image_for_finding_stars,
           directory_with_images=directory_with_images
           )

apass, vsx_apass_angle, targets_apass_angle = match(ccd, targets_from_file, vsx)

apass_good_coord, good_stars = mag_scale(Cmag, apass, vsx_apass_angle,
                                         targets_apass_angle,
                                         brighter_dmag=Cmag - bright_mag_limit,
                                         dimmer_dmag=dim_mag_limit - Cmag)

apass_comps = in_field(apass_good_coord, ccd, apass, good_stars)

box, iw = viewer()

make_markers(iw, ccd, targets_from_file, vsx, apass_comps,
             name_or_coord=coordinate)

box

### Get the table of marked stars.

In [ ]:
all_table = iw.get_all_markers()
elims = np.array([name.startswith('elim')
                 for name in all_table['marker name']])
elim_table = all_table[elims]
comp_table = all_table[~elims]

### Remove any that are marked for elimination as comparisons.

In [ ]:
index, d2d, d3d = elim_table['coord'].match_to_catalog_sky(comp_table['coord'])
comp_table.remove_rows(index)

### Sort the table

In [ ]:
if targets_from_file:
    target_coord = targets_from_file['coords'][0]
else:
    target_coord = SkyCoord(ra=ra_object, dec=dec_object, unit=("hour", "degree"))

# Calculate how far each is from target
comp_table['separation'] = target_coord.separation(comp_table['coord'])

# Add dummy column for sorting in the order we want
comp_table['sort'] = np.zeros(len(comp_table))

# Set sort order
apass_mark = comp_table['marker name'] == 'APASS comparison'
vsx_mark = comp_table['marker name'] == 'VSX'
tess_mark = ((comp_table['marker name'] == 'TESS Targets') | 
        (comp_table['separation'] < 0.3 * u.arcsec))


comp_table['sort'][apass_mark] = 2
comp_table['sort'][vsx_mark] = 1
comp_table['sort'][tess_mark] = 0

# Ensure the target is always first


# Sort the table
comp_table.sort(['sort', 'separation'])

# Assign the IDs
comp_table['id'] = range(1, len(comp_table) + 1)

### Display information on variables

In [ ]:
new_vsx_mark = comp_table['marker name'] == 'VSX'
vsx_dict = {}
idx, _, _ = comp_table['coord'][new_vsx_mark].match_to_catalog_sky(vsx['coords'])
for our_name, vsx_name in zip(comp_table['id'][new_vsx_mark], vsx['Name'][idx]):
    print(f'Our id number: {our_name}, VSX name: {vsx_name}')
    vsx_dict[f'V{our_name}'] = vsx_name

### Label the stars with numbers

In [ ]:
plot_names = []
for star in comp_table:
    star_id = star['id']
    if star['marker name'] == 'TESS Targets':
        label = f'T{star_id}'
        iw._marker = functools.partial(iw.dc.Text, text=label, fontsize=20, fontscale=False, color='green')
        iw.add_markers(Table(data=[[star['x']+20], [star['y']-20]], names=['x', 'y']))

    elif star['marker name'] == 'APASS comparison':
        label = f'C{star_id}'
        iw._marker = functools.partial(iw.dc.Text, text=label, fontsize=20, fontscale=False, color='red')
        iw.add_markers(Table(data=[[star['x']+20], [star['y']-20]], names=['x', 'y']))

    elif star['marker name'] == 'VSX':
        label = f'V{star_id}'
        iw._marker = functools.partial(iw.dc.Text, text=label, fontsize=20, fontscale=False, color='blue')
        iw.add_markers(Table(data=[[star['x']+20], [star['y']-20]], names=['x', 'y']))
    else:
        label = f'U{star_id}'
        print(f"Unrecognized marker name: {star['marker name']}")
    plot_names.append(label)

In [ ]:
comp_table['plot_label'] = [plot_names]

In [ ]:
comp_table.write(aperture_output_file, overwrite=True)

### Add 2.5 arcmin circle (do this after writing comparison table!)

Otherwise the comparison table will contain an extra entry for this circle.

In [ ]:
if targets_from_file:
    target = targets_from_file[0]
    iw.marker = {'color': 'yellow', 'radius': 268, 'type': 'circle'}
    iw.add_markers(target, skycoord_colname='coords',
                   use_skycoord=True, marker_name='target')